In [1]:
FileUtils.cd '../../'
require './config/boot'
APP_PATH  = File.expand_path('config/application') unless defined?(APP_PATH)
require APP_PATH
Rails.application.require_environment!

true

In [2]:
require "#{Rails.root}/app/models/subject"
require "#{Rails.root}/app/models/year_datum"
require "#{Rails.root}/app/models/faculty"
require "#{Rails.root}/app/models/subject_score"
require "#{Rails.root}/app/models/summarized_subject"

true

In [3]:
require 'open-uri'
require 'nokogiri'
require 'mechanize'
require 'robotex'

false

In [42]:
hash = Hash.new(0)

{}

## 設定

In [10]:
# 神学部 神学入門－3 
url = 'https://syllabus.doshisha.ac.jp/html/2016/03/103001003.html'

"https://syllabus.doshisha.ac.jp/html/2016/03/103001003.html"

In [54]:
# 文学 現代哲学（１）
url = 'https://syllabus.doshisha.ac.jp/html/2016/11/111079000.html'

"https://syllabus.doshisha.ac.jp/html/2016/11/111079000.html"

In [45]:
url = 'https://syllabus.doshisha.ac.jp/html/2015/37/137584004.html'

"https://syllabus.doshisha.ac.jp/html/2015/37/137584004.html"

In [55]:
agent = Mechanize.new
robotex = Robotex.new
p "robots: #{robotex.allowed?(url)}"
page = agent.get(url)
''

"robots: false"


""

## スクレイピング
タグ（項目？）ごとにtable, textのハッシュを作成

In [56]:
data = page.css('body > div > table:nth-child(2) > tbody > tr:nth-child(4) > td')
children=data.children
''

""

In [57]:
table_hash = {}
text_hash = {}
tag = 'init'

"init"

In [58]:
children.each do |t|
  text = t.text.strip
  if t.node_name == 'table'
    table_hash[tag] = t
  elsif text.present?
    if ret = text.match(/^＜(.+)＞$/)
      p "tag #{tag = ret[1]}"
    else
      text_hash[tag] = text
    end
  end
end
''
table_hash.keys

"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag 参考文献"


["授業計画", "成績評価基準", "参考文献"]

In [59]:
table_hash["成績評価基準"].css('tbody > tr > td:nth-child(1)').text

"\n平常点 \n\n期末試験 \n"

In [60]:
table_hash["成績評価基準"].css('tbody > tr > td:nth-child(2)').text

"\n30% \n\n70% \n"

In [61]:
table_hash["成績評価基準"].css('tbody > tr').each do |tr|
  tag = tr.css('td')[0].text.strip
  p tag
  hash[tag] += 1
end

"平常点 "
"期末試験 "


0

In [62]:
hash

{"平常点（出席，クラス参加，グループ作業の成果等） "=>1, "小レポート "=>1, "平常点(出席，クラス参加，グループ作業の成果等) "=>1, "平常点 "=>1, "期末試験 "=>1}

In [63]:
text_hash

{"概要"=>"本講義では、２０世紀における存在論の展開を取り上げます。アリストテレス以来、重要な哲学的主題の一つであり続けた存在論は、２０世紀に新たな展開をみます。ドイツでは、古典的存在論が、現象学的方法の発展と相まって深化を遂げ、フランスでは、異なる歴史的文脈から、このドイツの運動に新たな方向性がつけ加えられることになります。こうした２０世紀存在論の展開とそれに対するフランス現象学独自の貢献を、歴史的な背景を詳解しながら、明らかにしたいと思います。　　キーワード：存在、ハイデガー、実存論的分析論、レヴィナス、感覚", "到達目標"=>"２０世紀の主要な哲学的思考様式の一つを学び，現代哲学における存在論の意義と他の思想に与えた影響の広がりを把握することを目指します。", "授業計画"=>"授業計画は，受講生と相談の上で，適宜変更する場合があります。", "参考文献"=>"その他の関連文献については，各回に紹介します"}

## テキストから書籍名を取ってくる

In [64]:
t=table_hash["参考文献"].text

"\n\n\n\n\n\nハイデッガー \n『存在と時間（I, II, III）』原佑・渡邉二郎訳 (中公クラシックス、2003)\n\n\n\n\n\n \n\n\n\n\n\nレヴィナス \n『全体性と無限（上・下）』熊野純彦訳 (岩波文庫、2006)\n\n\n\n\n\n \n\n\n"

In [66]:
keyword = t.match(/『(.+)』/)[1]

"存在と時間（I, II, III）"

# Amazon URL
検索のキーワードを指定し，検索結果画面のリンクを作成する  
root: "https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=#{keyword}&rh=n:465392,k:#{keyword}&linkCode=ll2&tag=johshisha-22"

In [67]:
#keyword = "ゼロから始める"
url = "https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=#{keyword}&rh=n:465392,k:#{keyword}&linkCode=ll2&tag=johshisha-22"

"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=存在と時間（I, II, III）&rh=n:465392,k:存在と時間（I, II, III）&linkCode=ll2&tag=johshisha-22"